# Matching Algorithm

## Table of Content <a class="anchor" id="toc"></a>
#### [Summary](#overview)
#### Data
* [Globals](#globals)

#### Pre-Processing
* [Identification of Irrelevant Compounds](#irrelevant)
* [Identification of Cofactors](#cofactors)

#### Processing
* [Initialization of the Matching Algorithm](#eqcls)
* [Execution of the Matching Algorithm](#spawn)

#### Post-Processing
* [Match Characterization](#classify)
* [Direct Rule-Reaction Association](#rra)

## <a class="anchor" id="overview"></a>Summary [$\Uparrow$](#toc)

#### Pre-amble
The matching algorithm was built to find matches not only between database reactions and single predicted reactions, but also for sequences of predicted reactions. Furthermore, it matches reactions in both directions, i.e., assuming that the database reaction could also proceed in reverse direction. For completeness, and because it might be useful for readers in other contexts, we present the entire workflow. In the last cell of this notebook, reactions that are matched in forward direction by application of a single rule only are extracted, and this output is what is processed further for generating rule-enzyme links.

#### Projection of Substrates and Products to Equivalence Classes
Compounds are called equivalent if there is a chain of standardization steps that links one to the other. The assignment of equivalence classes to compounds is part of the initialisation process of the _DataProvider_ object defined in the _rra_-Python library.<br>
The matching algorithm then works not on the original database or predicted reactions but on formal reactions that turn equivalence classes of substrates into equivalence classes of products.

#### Match Finder
With the reactions' substrates and products projected onto equivalence classes and the non-essential compounds removed from the reaction equations, multigeneration biotransformation trees are built from the reactions predicted by the preceding [_in silico_ reaction](in%20silico%20reaction-KEGG.ipynb) step of the workflow. These start from  the substrate of a database reaction to be matched, and follow all the single prediction steps until there is no more rule-based reaction in the database starting from the compounds' equivalence classes that were last added to the tree. 

After the biotransformation tree has been built for a particular database reaction, the match finder tries to find paths from the  reactions' substrates to the reaction's products and vice versa.<br>
If any of the reactions' substrates or products cannot be linked in this way, this yields a "no match" outcome. If all of them can be linked, the algorithm looks for minimal sets of edges (predicted reactions) that, on the one hand, cover the full reaction, i.e. comprise all substrates and all products, and, on the other hand, do not overlap, i.e. each substrate and product is covered only once.<br> These minimal sets of edges in the biotransformation tree are considered a _match_. Matches that consist of a single edge (or predicted reaction) and are oriented in the same direction as the database reaction qualify for the last step of the workflow, the [rule-enzyme link](rule-enzyme%20link-KEGG.ipynb).

## <a class="anchor" id="globals"></a>Globals [$\Uparrow$](#toc)
#### The Configuration
Configuration data, such as credentials to access the worfklow database.

In [1]:
import yaml
with open("config.yaml", 'r') as stream:
    config = yaml.safe_load(stream)

#### Directories Used in the Workflow

In [2]:
DATASET = 'KEGG'
DATA = config['datadir'][DATASET]
TEMP = config['datadir']['temp']
BIN = config['binaries']
SRC = config['sources']

#### Workflow Database
Contains normalized SMILES, rules, reactions to be matched and the predicted reactions from the _in silico_ degradation.<br>
The content of the database is the main output from the first step of the workflow and the main input for the second step.

In [3]:
from sqlalchemy import create_engine
database = config['db']
DRIVER = database['driver']
HOST = database['host']
USER = database['user']
DB = database['name'][DATASET]
PW = database['password']
rradb = create_engine(f'{DRIVER}://{USER}:{PW}@{HOST}/{DB}')

## <a class="anchor" id="irrelevant"></a>Identification of Irrelevant Compounds [$\Uparrow$](#toc)

Molecules that do not contain carbon, nitrogen or sulfur atoms, and molecules with only one carbon, nitrogen or sulfur atom and no phosphorus atom are not further considered in the matching algorithm. This is because biodegradation rules from envipath are generally written in a form that they will not produce stoichiometrically balanced reaction equations, and often do not explicitely account for the formation of small cleavage products. Hence, in order to enable matching, they need to be removed from the reactions to be matched.

In [4]:
# compounds involved in unpredicted (data base) reactions
from pandas import read_sql
irrelevant_candidates = read_sql("""
    select count(*), compound, smiles
    from rrole join nsmiles on compound = id
    where reaction in (select reaction from edb_reaction)
    group by compound, smiles
    order by count(*) desc
""", rradb)

In [5]:
irrelevant_compounds = irrelevant_candidates\
[(irrelevant_candidates.smiles.str.count('[CNS]') == 0) | 
 ((irrelevant_candidates.smiles.str.count('[CNS]') == 1) & (irrelevant_candidates.smiles.str.count('P') == 0))]
print(irrelevant_compounds.shape)
irrelevant_compounds.head()

(61, 3)


,count,compound,smiles
0,2198,1,O
1,1650,6,[H+]
2,952,9,O=O
7,531,2,OP(=O)(O)OP(=O)(O)O
9,462,19,C(=O)=O


In [6]:
known_irrelevant = read_sql('select id from irrelevant', rradb)
irrelevant_compounds[~irrelevant_compounds['compound'].isin(known_irrelevant.id)]\
    .rename(columns={'compound':'id'})\
    .loc[:,['id']]\
    .to_sql('irrelevant', rradb, if_exists='append', method='multi', index=None)
read_sql('select id, smiles from irrelevant natural join nsmiles', rradb).shape

(61, 2)

## <a class="anchor" id="cofactors"></a>Identification of Cofactors  [$\Uparrow$](#toc)
The matching algorithm removes substrate-product pairs that are considered to be cofactors in the enzymatic reaction by declaration.
```
| Cofactors                | KEGG ids          |
| ------------------------ | ----------------- |
| NAD+ <-> NADH            | C00003 <-> C00004 |
| NADP+ <-> NADPH          | C00005 <-> C00006 |
| FAD <-> FADH2            | C00016 <-> C01352 |  
| ATP <-> ADP              | C00002 <-> C00008 |
| GTP <-> GDP              | C00044 <-> C00035 |
| UTP <-> UDP              | C00075 <-> C00015 |
```

In [7]:
from pandas import read_csv
cofactors = read_csv(f'{DATA}/../cofactors.standardized.tsv', sep="\t", header=None)
cofactors.columns = ['keggid', 'smiles', 'name']
cofactors['cofactor'] = [n.split(';')[0] for n in cofactors.name]
cofactors.loc[:,['cofactor', 'smiles']]

,cofactor,smiles
0,ATP,C([C@@H]1[C@H]([C@H]([C@H](N2C=NC3=C2N=CN=C3N)...
1,ADP,C([C@@H]1[C@H]([C@H]([C@H](N2C=NC3=C2N=CN=C3N)...
2,NADH,C1=CN(C=C(C1)C(=O)N)[C@H]2[C@@H]([C@@H]([C@@H]...
3,NAD+,C1=C[N+](=CC(=C1)C(=O)N)[C@H]2[C@@H]([C@@H]([C...
4,NADP+,C1=C[N+](=CC(=C1)C(=O)N)[C@H]2[C@@H]([C@@H]([C...
5,NADPH,C1=CN(C=C(C1)C(=O)N)[C@H]2[C@@H]([C@@H]([C@@H]...
6,UDP,C1=CN([C@H]2[C@@H]([C@@H]([C@@H](COP(=O)(O)OP(...
7,UTP,C1=CN([C@H]2[C@@H]([C@@H]([C@@H](COP(=O)(O)OP(...
8,FAD,CC1=CC2=C(C=C1C)N(C[C@@H]([C@@H]([C@@H](COP(=O...
9,GDP,C([C@@H]1[C@H]([C@H]([C@H](N2C=NC3=C2N=C(N)NC3...


In [8]:
from pandas import DataFrame
cofactors_id = cofactors.merge(read_sql('select id, smiles from nsmiles', rradb), on='smiles').loc[:,['cofactor','id']]

cofactor_pairs = [
    ['NAD+', 'NADH'],
    ['NADP+', 'NADPH'],
    ['FAD', 'FADH2'],
    ['ATP', 'ADP'],
    ['GTP', 'GDP'],
    ['UTP', 'UDP'],
]
cofactor_pairs = DataFrame(cofactor_pairs, columns=['cofactor_a', 'cofactor_b'])
cofactor_pairs_id = cofactor_pairs\
    .merge(cofactors_id.rename(columns={'cofactor':'cofactor_a'}), on='cofactor_a').rename(columns={'id':'a'})\
    .merge(cofactors_id.rename(columns={'cofactor':'cofactor_b'}), on='cofactor_b').rename(columns={'id':'b'})
cofactor_pairs_id

,cofactor_a,cofactor_b,a,b
0,NAD+,NADH,33,32
1,NADP+,NADPH,36,37
2,FAD,FADH2,63,354
3,ATP,ADP,26,28
4,GTP,GDP,153,151
5,UTP,UDP,62,60


In [9]:
rradb.execute('delete from transition')
cofactor_pairs_id.loc[:,['a','b']].to_sql('transition', rradb, index=None, if_exists='append', method='multi')
read_sql('select * from transition', rradb)

,a,b
0,33,32
1,36,37
2,63,354
3,26,28
4,153,151
5,62,60


## <a class="anchor" id="eqcls"></a>Initialization of the Matching Algorithm [$\Uparrow$](#toc)

By creating a `DataProvider` object, the matching algorithm is initialized.

In [10]:
import sys
sys.path.append(f'{SRC}')
from rra import DataProvider
dp = DataProvider(
    dbname=DB,
    driver=DRIVER,
    host=HOST,
    user=USER,
    password=PW,
    standardizer_ids=None
)

During the creation of a `DataProvider` object, the reactions from the database are transformed into "meta reactions" where both, substrates and products, are not single molecules but equivalence classes of molecules. Depending on the size of the database this transformation can take a while.<br>
Also, non-essential compounds, i.e. irrelevant compounds and co-factors, are removed from the reactions during the object creation.

<b>Remark:</b> By explicitly listing the standardizers' ids in the constructor's argument, it is possible to filter for the standardizers upon which the equivalence relationship is based. If the argument is omitted, all standardizers are considered.

## <a class="anchor" id="spawn"></a>Execution of the Matching Algorithm [$\Uparrow$](#toc)

The matching algorithm is implemented in the _rra_ Python library and is run by calling the <i>match_reaction</i> method thereof for each reaction of the dataset.

In [11]:
dataset_reactions = read_sql('''
select reaction, envipath_url
from edb_reaction
''', rradb)
dataset_reactions.head()

,reaction,envipath_url
0,1,R00004
1,2,R00011
2,3,R00025
3,4,R00033
4,5,R00072


In [12]:
from rra import match_reaction

matches = {}
matchexceptions = {}

def fill_matches(reac_id):
    try:
        reaction_matches = list(match_reaction(dp, reac_id, step_max=1, backward_hitlimit=300, forward_hitlimit=300, suppress_warnings=True))
        if reaction_matches:
            matches[reac_id] = reaction_matches
    except Exception as e:
        matchexceptions[reac_id] = e

from tqdm import tqdm
tqdm.pandas()
dataset_reactions.progress_apply(lambda row: fill_matches(row.reaction), axis=1)

with open(f'{TEMP}/matches.dump', 'w') as matchdump:
    matchdump.write(f'{matches}')

100%|██████████| 6584/6584 [23:26<00:00,  2.27it/s]  


In [13]:
try:
    print(len(matches.keys()))
except:
    import ast
    with open(f'{TEMP}/matches.dump') as dump:
        matches = ast.literal_eval(dump.read())

3222


## <a class="anchor" id="classify"></a>Match Characterization [$\Uparrow$](#toc)

The results of the match finding algorithm are compiled into a dataframe holding the reaction's identifier, all predicted paths that lead to a match and the respective rules involved in these prediction paths. Furthermore, the matches are characterized by the direction of the matching prediction path relative to the direction of the original reaction, e.g. if the reaction <code>A > B</code> is triggered by rule `R1` and <code>B > A</code> is triggered by rule `R2`, the corresponding reaction row will have two sets of involved rules: `(f,[R1])` and `(r,[R2])` where _f<a></a>_ stands for 'forward' and _r_ for 'reverse'.

In [14]:
from rra.logic import sufficient_rule_sets, non_reducible_rule_sets

matches_df = DataFrame([{'rid':rid, 'matches': matches[rid]} for rid in matches])
matches_df.reset_index(drop=True, inplace=True)
matches_df['rulesets'] = matches_df[matches_df.matches.notnull()].progress_apply(
    lambda row: non_reducible_rule_sets(sufficient_rule_sets(dp, row.matches, coln='simpfix')),
    axis=1
);
matches_df.to_pickle(f'{DATA}/matches_df_plusd.pkl')
matches_df.head()

100%|██████████| 3222/3222 [00:11<00:00, 280.36it/s]


,matches,rid,rulesets
0,"[[(True, [70495])]]",4,"[(f, [3568])]"
1,"[[(True, [108952])]]",5,"[(f, [1196])]"
2,"[[(True, [123645])], [(True, [48902])], [(True...",6,"[(f, [4141])]"
3,"[[(True, [123646])], [(True, [48904])], [(True...",7,"[(f, [4141])]"
4,"[[(True, [123645])], [(True, [48902])], [(True...",8,"[(f, [4141])]"


## <a class="anchor" id="rra"></a>Direct Rule-Reaction Association [$\Uparrow$](#toc)

The reactions that are matched by applying a single rule to their substrates are directly associated with that rule and collected into a file. This file contains the essential outcome of the algorithm that is further processed for generating rule-enzyme links.<br>

In [15]:
def unambigous_associations(df):
    for i in df.index:
        if df.loc[i, 'rulesets'] != df.loc[i, 'rulesets']:
            continue
        for (direction,ruleset) in df.loc[i,'rulesets']:
            if len(ruleset) == 1 and direction == 'f':
                yield {'reaction':df.loc[i,'rid'], 'bt':ruleset[0], 'direction':direction}

direct_links = DataFrame(unambigous_associations(matches_df))\
    .merge(dataset_reactions, on='reaction')\
    .drop_duplicates()

direct_links.to_csv(f'{DATA}/direct_links.tsv', index=None, sep="\t")

print(direct_links.shape)
direct_links.tail()

(2003, 4)


,bt,direction,reaction,envipath_url
1998,3564,f,6558,R12623
1999,3564,f,6559,R12624
2000,3564,f,6560,R12625
2001,4013,f,6564,R12635
2002,4141,f,6568,R12641
